In [16]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

import warnings
warnings.filterwarnings('ignore')

In [18]:
train_dataset, val_dataset, train_loader, val_loader = CIFAR100()

Files already downloaded and verified
Files already downloaded and verified


In [19]:
# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"



# Train shallow CNN on CIFAR 100

In [20]:
from torch.utils.tensorboard import SummaryWriter
net_ori = BaseNet_test().to(device)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_ori.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar100_base.pth'
device = "cuda"
writer = SummaryWriter("NormalBase")

In [21]:
train(net_ori, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 4.605538014560709	 lr: 0.001
Epoch [1/100]	 Validation Loss: 4.605128451238705	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 4.605038394098696	 lr: 0.001
Epoch [2/100]	 Validation Loss: 4.604664355893679	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 4.604372770889945	 lr: 0.001
Epoch [3/100]	 Validation Loss: 4.603858802891985	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 4.6033164656070795	 lr: 0.001
Epoch [4/100]	 Validation Loss: 4.602409718911859	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 4.601454359186275	 lr: 0.001
Epoch [5/100]	 Validation Loss: 4.599879832207402	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 4.597498498609304	 lr: 0.001
Epoch [6/100]	 Validation Loss: 4.593843870525118	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 4.5875228725735795	 lr: 0.001
Epoch [7/100]	 Validation Loss: 4.576691627502441	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 4.548379

Epoch [60/100]	 Training Loss: 1.4697922270011414	 lr: 0.001
Epoch [60/100]	 Validation Loss: 3.7062741020057777	 lr: 0.001
Epoch [60/100]	
Epoch [61/100]	 Training Loss: 1.4087214494300315	 lr: 0.001
Epoch [61/100]	 Validation Loss: 3.7080571711817876	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 1.3511397003212853	 lr: 0.001
Epoch [62/100]	 Validation Loss: 3.806745447689974	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 1.2907610896908108	 lr: 0.001
Epoch [63/100]	 Validation Loss: 3.906908835036845	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.2256876899458258	 lr: 0.001
Epoch [64/100]	 Validation Loss: 3.91516763047327	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.1737164242188338	 lr: 0.001
Epoch [65/100]	 Validation Loss: 4.077345169043239	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.1084116261328578	 lr: 0.001
Epoch [66/100]	 Validation Loss: 4.163233548780031	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training L

In [22]:
#test clean accuray
test(val_loader, net_ori)


Accuracy of the network on the 10000 test images: 27 %


In [23]:
print("PGD")
pgd_attack_transfer = generate_adv(net_ori, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_ori, "cuda")

PGD
[Model loaded]
Acc: 1.17 %
adv model


In [24]:
print("fgsm")
fgsm_attack_transfer = generate_adv(net_ori, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_ori, "cuda")

fgsm
[Model loaded]
Acc: 2.56 %
adv model


In [25]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(net_ori, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_ori, "cuda")

nifgsm
[Model loaded]
Acc: 1.77 %
adv model


In [26]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(net_ori, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_ori, "cuda")

vmifgsm
[Model loaded]
Acc: 1.39 %
adv model


# Adversarial Training on PGD

In [36]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(net_ori) 
for param in net_adv.parameters():
    print(param)

Parameter containing:
tensor([[[[ 6.8848e-01,  7.1635e-01,  8.7562e-01,  7.3762e-01,  7.1002e-01],
          [ 9.6923e-01,  9.3511e-01,  1.1260e+00,  1.0334e+00,  8.8704e-01],
          [ 1.0669e+00,  1.2664e+00,  1.2851e+00,  1.2665e+00,  1.0994e+00],
          [ 8.0321e-01,  1.0094e+00,  1.3014e+00,  1.2376e+00,  1.0452e+00],
          [ 6.0307e-01,  8.9163e-01,  1.0605e+00,  1.0001e+00,  7.4786e-01]],

         [[-4.6893e-02,  7.6766e-02,  1.7172e-01,  7.9294e-02,  1.1587e-01],
          [ 2.4507e-01,  3.0737e-01,  3.8427e-01,  3.0031e-01,  2.2433e-01],
          [ 2.9689e-01,  4.6346e-01,  4.3376e-01,  2.7984e-01,  3.9987e-01],
          [ 3.6214e-02,  2.0558e-01,  3.0059e-01,  2.3015e-01,  2.2317e-01],
          [-1.4972e-01,  9.5225e-02,  8.3450e-02,  1.7958e-01,  1.7341e-01]],

         [[-3.9027e-01, -3.6181e-01, -3.4574e-01, -1.9384e-01, -2.1003e-01],
          [-1.0891e-01,  6.7987e-02,  3.7039e-02, -1.8180e-02, -1.7407e-01],
          [-1.2289e-01,  1.4269e-01,  1.9234e-01, 

In [37]:
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_pgd_3.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

In [29]:
train(net_adv, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 5.61242687610714	 lr: 0.001
Epoch [1/100]	 Validation Loss: 4.052706042422524	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 4.524453789986613	 lr: 0.001
Epoch [2/100]	 Validation Loss: 3.8696791914444937	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 4.373876644827216	 lr: 0.001
Epoch [3/100]	 Validation Loss: 3.7542997130864784	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 4.277758375153212	 lr: 0.001
Epoch [4/100]	 Validation Loss: 3.6750665797462947	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 4.204787092745456	 lr: 0.001
Epoch [5/100]	 Validation Loss: 3.6098694016661823	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 4.150112204539502	 lr: 0.001
Epoch [6/100]	 Validation Loss: 3.5501100262509118	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 4.106544750730705	 lr: 0.001
Epoch [7/100]	 Validation Loss: 3.528781199757057	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 4.0671

Epoch [61/100]	 Training Loss: 3.19021529919656	 lr: 0.001
Epoch [61/100]	 Validation Loss: 3.1020521604562106	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 3.1720830912480267	 lr: 0.001
Epoch [62/100]	 Validation Loss: 3.0996938023386122	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 3.160540592335069	 lr: 0.001
Epoch [63/100]	 Validation Loss: 3.1083232179472717	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 3.147827190511367	 lr: 0.001
Epoch [64/100]	 Validation Loss: 3.1087988328330125	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 3.1325503932240673	 lr: 0.001
Epoch [65/100]	 Validation Loss: 3.1053486685209637	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 3.1196492821969035	 lr: 0.001
Epoch [66/100]	 Validation Loss: 3.117337564878826	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training Loss: 3.108081431644957	 lr: 0.001
Epoch [67/100]	 Validation Loss: 3.1088086774077595	 lr: 0.001
Epoch [67/100]	
Epoch [68/100]	 Training L

In [30]:
#test clean accuray
test(val_loader, net_adv)

Accuracy of the network on the 10000 test images: 27 %


In [32]:
print("PGD")
pgd_attack_transfer = generate_adv(net_ori, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_adv, "cuda")



PGD
[Model loaded]
Acc: 19.96 %
adv model


In [33]:
print("fgsm")
fgsm_attack_transfer = generate_adv(net_ori, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_adv, "cuda")



fgsm
[Model loaded]
Acc: 19.88 %
adv model


In [34]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(net_ori, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_adv, "cuda")



nifgsm
[Model loaded]
Acc: 20.20 %
adv model


In [35]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(net_ori, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_adv, "cuda")

vmifgsm
[Model loaded]
Acc: 19.85 %
adv model


# TODO: Adversarial Training by nifgsm

In [38]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv_ni = copy.deepcopy(net_ori) #am I doing correct? 

train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv_ni.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_nifgsm.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_nifgsm")

ni_attack = generate_adv(net_adv_ni, "nifgsm")

train(net_adv_ni, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, True, ni_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 5.406936675996122	 lr: 0.001
Epoch [1/100]	 Validation Loss: 3.9515698620035677	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 4.428898472310332	 lr: 0.001
Epoch [2/100]	 Validation Loss: 3.7453821007209487	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 4.261738850332587	 lr: 0.001
Epoch [3/100]	 Validation Loss: 3.6030167235603816	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 4.144184082670285	 lr: 0.001
Epoch [4/100]	 Validation Loss: 3.52122652681568	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 4.051571045995064	 lr: 0.001
Epoch [5/100]	 Validation Loss: 3.4617975633355638	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 3.9700740932503624	 lr: 0.001
Epoch [6/100]	 Validation Loss: 3.415811061859131	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 3.899870818838134	 lr: 0.001
Epoch [7/100]	 Validation Loss: 3.3736180746102633	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 3.835

Epoch [61/100]	 Training Loss: 2.0421513236697066	 lr: 0.001
Epoch [61/100]	 Validation Loss: 3.6591210154038443	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 2.022821721213553	 lr: 0.001
Epoch [62/100]	 Validation Loss: 3.6626508779163602	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 2.002176820164751	 lr: 0.001
Epoch [63/100]	 Validation Loss: 3.7089401317548147	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.9826039868547483	 lr: 0.001
Epoch [64/100]	 Validation Loss: 3.700410353986523	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.9654374704946338	 lr: 0.001
Epoch [65/100]	 Validation Loss: 3.694437078282803	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.9439314780637735	 lr: 0.001
Epoch [66/100]	 Validation Loss: 3.7472692169720614	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training Loss: 1.9276827305479123	 lr: 0.001
Epoch [67/100]	 Validation Loss: 3.7558007210115845	 lr: 0.001
Epoch [67/100]	
Epoch [68/100]	 Training

In [39]:
#test clean accuray
test(val_loader, net_adv_ni)

Accuracy of the network on the 10000 test images: 26 %


In [40]:
print("PGD")
pgd_attack_transfer = generate_adv(net_ori, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_adv_ni, "cuda")



PGD
[Model loaded]
Acc: 17.50 %
adv model


In [41]:
print("fgsm")
fgsm_attack_transfer = generate_adv(net_ori, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_adv_ni, "cuda")



fgsm
[Model loaded]
Acc: 17.41 %
adv model


In [42]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(net_ori, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_adv_ni, "cuda")



nifgsm
[Model loaded]
Acc: 17.63 %
adv model


In [43]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(net_ori, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_adv_ni, "cuda")

vmifgsm
[Model loaded]
Acc: 17.17 %
adv model


# TODO: Adversarial Training by vmifgsm

In [44]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_vmifgsm = copy.deepcopy(net_ori) 

train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_vmifgsm.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_vmifgsm.pth'
device = "cuda"

writer = SummaryWriter("Adv_Base_vmifgsm")
vmi_attack = generate_adv(net_vmifgsm, "vmifgsm")

train(net_vmifgsm, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, True, vmi_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 5.593234006096335	 lr: 0.001
Epoch [1/100]	 Validation Loss: 4.114244599885579	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 4.539997804500258	 lr: 0.001
Epoch [2/100]	 Validation Loss: 3.916336949867538	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 4.402561780436875	 lr: 0.001
Epoch [3/100]	 Validation Loss: 3.8532697339601154	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 4.308990769983862	 lr: 0.001
Epoch [4/100]	 Validation Loss: 3.7676802586905564	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 4.240994041837999	 lr: 0.001
Epoch [5/100]	 Validation Loss: 3.6924394112599046	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 4.188209241613403	 lr: 0.001
Epoch [6/100]	 Validation Loss: 3.635173864002469	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 4.142478986164493	 lr: 0.001
Epoch [7/100]	 Validation Loss: 3.5910115423081796	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 4.1043

Epoch [61/100]	 Training Loss: 3.184679027103707	 lr: 0.001
Epoch [61/100]	 Validation Loss: 3.1444551642936998	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 3.1722247923731497	 lr: 0.001
Epoch [62/100]	 Validation Loss: 3.149228729779207	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 3.1564481386443233	 lr: 0.001
Epoch [63/100]	 Validation Loss: 3.1411635785163203	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 3.141174600557293	 lr: 0.001
Epoch [64/100]	 Validation Loss: 3.144280711306801	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 3.126172642573676	 lr: 0.001
Epoch [65/100]	 Validation Loss: 3.162195260011697	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 3.1116788454372863	 lr: 0.001
Epoch [66/100]	 Validation Loss: 3.145811784116528	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training Loss: 3.098067507414562	 lr: 0.001
Epoch [67/100]	 Validation Loss: 3.1461163744141785	 lr: 0.001
Epoch [67/100]	
Epoch [68/100]	 Training Los

In [45]:
#test clean accuray
test(val_loader, net_vmifgsm)

Accuracy of the network on the 10000 test images: 26 %


In [46]:
print("PGD")
pgd_attack_transfer = generate_adv(net_ori, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_vmifgsm, "cuda")



PGD
[Model loaded]
Acc: 20.23 %
adv model


In [47]:
print("fgsm")
fgsm_attack_transfer = generate_adv(net_ori, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_vmifgsm, "cuda")



fgsm
[Model loaded]
Acc: 20.03 %
adv model


In [48]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(net_ori, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_vmifgsm, "cuda")



nifgsm
[Model loaded]
Acc: 20.30 %
adv model


In [49]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(net_ori, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_vmifgsm, "cuda")

vmifgsm
[Model loaded]
Acc: 19.97 %
adv model
